<a href="https://colab.research.google.com/github/ashwinvaswani/whatif/blob/master/Code/CL_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import all necessary functions and write the helper functions to extract the text and clean it from the url

In [0]:
import nltk
import numpy as np
import scipy
import sklearn
import requests
import bs4 as bs
import re,string
import pandas as pd
import requests

In [0]:
def clean(s_l):
    cl_s = ""
    for i in s_l:
        if(i == ''):
            continue
        s = re.sub('[\W_]+', ' ', i)
        cl_s += s + " "
    return cl_s

In [0]:
def getText(html):
    try:
      soup = bs.BeautifulSoup(html.content,'html.parser')
      st = []
      for paragraph in soup.find_all('p'):
          st.append(str(paragraph.text).strip())
      return clean(st)
    except:
      print("ERROR IN TEXT FOR THE WEBSITE")

#Get the Form Index

In [0]:
    matches_form = []
    venues = []
    teams = []
    dates = []

In [0]:
season_14_15 = ["https://globalsportsarchive.com/competition/soccer/uefa-champions-league-2014-2015/final/3856/",
                  "https://globalsportsarchive.com/competition/soccer/uefa-champions-league-2014-2015/semi-finals/3855/",
                  "https://globalsportsarchive.com/competition/soccer/uefa-champions-league-2014-2015/quarter-finals/3854/",
                  "https://globalsportsarchive.com/competition/soccer/uefa-champions-league-2014-2015/round-of-16/3853/"]

season_15_16 = ["https://globalsportsarchive.com/competition/soccer/uefa-champions-league-2015-2016/final/6171/",
"https://globalsportsarchive.com/competition/soccer/uefa-champions-league-2015-2016/semi-finals/6169/",
"https://globalsportsarchive.com/competition/soccer/uefa-champions-league-2015-2016/quarter-finals/6168/",
"https://globalsportsarchive.com/competition/soccer/uefa-champions-league-2015-2016/round-of-16/6167/"]

season_16_17 = ["https://globalsportsarchive.com/competition/soccer/uefa-champions-league-2016-2017/final/8237/",
"https://globalsportsarchive.com/competition/soccer/uefa-champions-league-2016-2017/semi-finals/8236/",
"https://globalsportsarchive.com/competition/soccer/uefa-champions-league-2016-2017/quarter-finals/8235/",
"https://globalsportsarchive.com/competition/soccer/uefa-champions-league-2016-2017/round-of-16/8234/"]

season_17_18 = ["https://globalsportsarchive.com/competition/soccer/uefa-champions-league-2017-2018/final/13427/",
"https://globalsportsarchive.com/competition/soccer/uefa-champions-league-2017-2018/semi-finals/13426/",
"https://globalsportsarchive.com/competition/soccer/uefa-champions-league-2017-2018/quarter-finals/13425/",
"https://globalsportsarchive.com/competition/soccer/uefa-champions-league-2017-2018/round-of-16/13424/"]

season_18_19 = ["https://globalsportsarchive.com/competition/soccer/uefa-champions-league-2018-2019/final/22279/",
"https://globalsportsarchive.com/competition/soccer/uefa-champions-league-2018-2019/semi-finals/22278/",
"https://globalsportsarchive.com/competition/soccer/uefa-champions-league-2018-2019/quarter-finals/22277/",
"https://globalsportsarchive.com/competition/soccer/uefa-champions-league-2018-2019/round-of-16/22276/"]

seasons = [season_14_15, season_15_16, season_16_17, season_17_18, season_18_19]

In [0]:
    season_19_20 = "https://globalsportsarchive.com/competition/soccer/uefa-champions-league-2019-2020/round-of-16/32402/"
    headers = {'user-agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/80.0.3987.87 Chrome/80.0.3987.87 Safari/537.36'}
    page = requests.get(season_19_20,headers=headers)
    if(page.status_code == 200):
            soup = bs.BeautifulSoup(page.content, "html.parser")


    matchs_2019 = [(x['href'].split('/')[5],x['href']) 
                  for x in soup.find_all('a',{'title':re.compile('match report.*')})
                  if not x.find('span',{'class':'match_score'}).text == 'PSTP']
    matchs_2019 = sorted(matchs_2019,reverse=True)
    match_links = matchs_2019
    for link in match_links:    
        #headers = {'user-agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/80.0.3987.87 Chrome/80.0.3987.87 Safari/537.36'}
        dates.append(link[0])
        dates.append(link[0])
        page = requests.get(link[1],headers=headers)
        if(page.status_code == 200):
            soup = bs.BeautifulSoup(page.content, "html.parser")


        home = soup.find('div',{'id':"match_header_team_a"}).find('span',class_="name_full").text.replace("FC",'')
        away = soup.find('div',{'id':"match_header_team_b"}).find('span',class_="name_full").text.replace("FC",'')
        teams.append(home)
        teams.append(away)
        venue = soup.find_all('div',{'class':'info_2'})[1].text
        venues.append(venue)
        venues.append(venue)

        matches = soup.find('div',{'class':"dsg-w-8-b-c dsg-w-8-matches"}).find_all('div',{'class':'dsg-w-8-col-1'})
        home_form_team = []
        away_form_team = []

        home_forms = matches[0].find_all('div',{'class':'dsg-w-8-m-row tal'})
        away_forms = matches[1].find_all('div',{'class':'dsg-w-8-m-row tbl'})

        for form in home_forms:
            if (form.find('div',{'class':'bold color-win'})):
                home_form_team.append("W")
            elif (form.find('div',{'class':'bold color-draw'})):
                home_form_team.append("D")
            else:
                home_form_team.append("L")

        for form in away_forms:
            if (form.find('div',{'class':'bold color-win'})):
                away_form_team.append("W")
            elif (form.find('div',{'class':'bold color-draw'})):
                away_form_team.append("D")
            else:
                away_form_team.append("L")

        matches_form.append(home_form_team)
        matches_form.append(away_form_team)


In [0]:
df = pd.DataFrame()
df['Date'] = dates
df['Teams'] = teams
df['Venue'] = venues
df['Form'] = matches_form



In [0]:
df.to_csv("team_form_final.csv",index=False)

#Obtain the data for the 2019 Champions League

In [0]:
def get_dataset_2019(url,url2):
  page = requests.get(url)
  if(page.status_code == 200):
      soup = bs.BeautifulSoup(page.content, "html.parser")
  page2 = requests.get(url2)
  if(page2.status_code == 200):
      soup2 = bs.BeautifulSoup(page2.content, "html.parser")

  # Getting the score
  team_home_score = soup.find('span', {'class' : 'js-team--home-score'}).text
  team_away_score = soup.find('span', {'class' : 'js-team--away-score'}).text
  
  try:
    penal_score = soup.find('div', {'class' : 'js-match-status-aggr match-status-aggr'}).text
    temp = penal_score.split()
    home_penal = int(temp[1][0])
    away_penal = int(temp[1][-1])
  except:
    home_penal = 0
    away_penal = 0
#Getting team names
  temp = soup.find('div',{'class':"match-row_match d3-plugin"})
  try:
      team_home_name = temp.find('div', {'class' : 'team-home is-club match--winner'}).find('div', {'class' : 'team-name'}).find("div").find('span', {'class' : 'js-fitty'}).text
  except:
      team_home_name = temp.find('div', {'class' : 'team-home is-club '}).find('div', {'class' : 'team-name'}).find("div").find('span', {'class' : 'js-fitty'}).text
  team_home_name = team_home_name.strip()
  

  try:
      team_away_name = temp.find('div', {'class' : 'team-away is-club match--winner'}).find('div', {'class' : 'team-name'}).find("div").find('span', {'class' : 'js-fitty'}).text
  except:
      team_away_name = temp.find('div', {'class' : 'team-away is-club '}).find('div', {'class' : 'team-name'}).find("div").find('span', {'class' : 'js-fitty'}).text
  team_away_name = team_away_name.strip()
  
# Getting the scorers
  home_scorers_list = []
  temp = soup.find("ul", {'class':'match--scorers js-home-team--scorers'})
  scorers = temp.find_all("li", {'class':'scorer'})

  for i in scorers:
      home_scorers_list.append(i.text.strip())

  away_scorers_list = []
  temp = soup.find("ul", {'class':'match--scorers js-away-team--scorers'})
  scorers = temp.find_all("li", {'class':'scorer'})

  for i in scorers:
      away_scorers_list.append(i.text.strip())

  home_scorer_name ={}
  for i in home_scorers_list:
      h = i.split()
      name = ""
      goal_index = 0
      for s in h:
          if(s[0].isdigit()):
              break
          goal_index +=1
          name+= s + " "
      name = name.strip()
      home_scorer_name[name] = []
      home_scorer_name[name] = h[goal_index:]

  away_scorer_name ={}
  for i in away_scorers_list:
      h = i.split()
      name = ""
      goal_index = 0
      for s in h:
          if(s[0].isdigit()):
              break
          goal_index +=1
          name+= s + " "
      name = name.strip()
      away_scorer_name[name] = []
      away_scorer_name[name] = h[goal_index:]

#Getting location
  stadium_name = soup.find("span", {"class":"match-location_stadium"}).text
  stadium_city = soup.find("span", {"class":"match-location_venue"}).text
  location = stadium_name + ", " + stadium_city

#Getting player names
  url_lineups = url + 'lineups/?iv=true'
  page = requests.get(url_lineups)
  if(page.status_code == 200):
      soup = bs.BeautifulSoup(page.content, "html.parser")
#Home players  
  home_players = soup.find_all("li", {"class":"squad--team-player squad--team-home squad--team-home-player"})

  home_players_list = []
  for i in home_players:
      a = i.find("span", {"class": "squad--player-name"}).find("div").find('span', {'class' : 'js-fitty'}).text
      home_players_list.append(a.strip())

  home_players_list = list(dict.fromkeys(home_players_list))
  coach = home_players_list[-1]
  home_players_list.pop()

  home_players_numbers = []
  for i in home_players:
    a = i.find('span',{'class':'squad--player-num'})
    if a == None:
      continue
    else:
      home_players_numbers.append(a.text.strip())
  home_players_numbers = list(dict.fromkeys(home_players_numbers))

#Away players
  away_players = soup.find_all("li", {"class":"squad--team-player squad--team-away squad--team-away-player"})

  away_players_list = []
  for i in away_players:
      a = i.find("span", {"class": "squad--player-name"}).find("div").find('span', {'class' : 'js-fitty'}).text
      away_players_list.append(a.strip())

  away_players_list = list(dict.fromkeys(away_players_list))
  
  coach_away = away_players_list[-1]
  away_players_list.pop()

  away_players_numbers = []
  for i in away_players:
    a = i.find('span',{'class':'squad--player-num'})
    if a == None:
      continue
    else:
      away_players_numbers.append(a.text.strip())
  away_players_numbers = list(dict.fromkeys(away_players_numbers))

  count = 0
  started_home = []
  bench_home = []
  for i in home_players_list:
      if("(GK)" in i):
          count+=1
      if(count < 2):
          started_home.append(i)
      else:
          bench_home.append(i)

  count = 0
  started_away = []
  bench_away = []
  for i in away_players_list:
      if("(GK)" in i):
          count+=1
      if(count < 2):
          started_away.append(i)
      else:
          bench_away.append(i)
#substitutions 
  substitutions_home = []
  for i in home_players:
      name = i.find("span", {"class": "squad--player-name"}).find("div").find('span', {'class' : 'js-fitty'}).text
      name=" ".join(name.split())
      
      a = i.find("span", {"class":"lineups--events"})
      if(a==None):
          continue
      b = a.find_all("img", alt=True)
      for img in b:
          if(img["alt"] == "Substitution"):
              g = img.findNext("span").text
              substitutions_home.append((name,g))

  substitutions_home = list(dict.fromkeys(substitutions_home))
  

  substitutions_away = []
  for i in away_players:
      name = i.find("span", {"class": "squad--player-name"}).find("div").find('span', {'class' : 'js-fitty'}).text
      name=" ".join(name.split())
      
      a = i.find("span", {"class":"lineups--events"})
      if(a==None):
          continue
      b = a.find_all("img", alt=True)
      for img in b:
          if(img["alt"] == "Substitution"):
              g = img.findNext("span").text
              substitutions_away.append((name,g))

  substitutions_away = list(dict.fromkeys(substitutions_away))
  

  l = len(substitutions_home)
  l = l//2
  did_not_finish_home = substitutions_home[:l]
  

  joined_game_home = substitutions_home[l:]
  

  l = len(substitutions_away)
  l = l//2
  did_not_finish_away = substitutions_away[:l]
  

  joined_game_away = substitutions_away[l:]

#Player stats from FBREF
  code_links = soup2.find_all('img',{'class':'teamlogo'})[:2]
  home_code = code_links[0]['src'].split('/')[-1].replace('.png','')
  away_code = code_links[1]['src'].split('/')[-1].replace('.png','')
  year = int(url2.split('-')[-3])

  home_stats_list = []
  if year <= 2018:
    temp = soup2.find('table',id='stats_'+home_code+'_summary').find('tbody')
    for i in temp:
      if i.find('td') == -1:
        continue
      number = i.find('td',{'data-stat':'shirtnumber'}).text
      fouls = i.find('td',{'data-stat':'fouls'}).text     #2
      assists = i.find('td',{'data-stat':'assists'}).text #3
      offside = i.find('td',{'data-stat':'offsides'}).text #4
      interceptions = i.find('td',{'data-stat':'interceptions'}).text #5
      crosses = i.find('td',{'data-stat':'crosses'}).text #6
      shots = i.find('td',{'data-stat':'shots_total'}).text
      shots_on_target = i.find('td',{'data-stat':'shots_on_target'}).text
      home_stats_list.append(list((number,fouls,assists,offside,interceptions,crosses,shots,shots_on_target)))
    
    

  else:
    temp = soup2.find('table',id='stats_'+home_code+'_summary').find('tbody')
    temp2 = soup2.find('table',id='stats_'+home_code+'_misc').find('tbody')
    for i,j in zip(temp,temp2):
      if i.find('td') == -1 or j.find('td') == -1:
        continue
      
      number = i.find('td',{'data-stat':'shirtnumber'}).text
      fouls = j.find('td',{'data-stat':'fouls'}).text     #2
      assists = i.find('td',{'data-stat':'assists'}).text #3
      offside = j.find('td',{'data-stat':'offsides'}).text #4
      interceptions = j.find('td',{'data-stat':'interceptions'}).text #5
      crosses = j.find('td',{'data-stat':'crosses'}).text #6
      shots = i.find('td',{'data-stat':'shots_total'}).text
      shots_on_target = i.find('td',{'data-stat':'shots_on_target'}).text
      home_stats_list.append(list((number,fouls,assists,offside,interceptions,crosses,shots,shots_on_target)))
    
  
  
  away_stats_list = []
  if year <=2018:
    temp = soup2.find('table',id='stats_'+away_code+'_summary').find('tbody')
    for i in temp:
      if i.find('td') == -1:
        continue
      
      number = i.find('td',{'data-stat':'shirtnumber'}).text
      fouls = i.find('td',{'data-stat':'fouls'}).text     #2
      assists = i.find('td',{'data-stat':'assists'}).text #3
      offside = i.find('td',{'data-stat':'offsides'}).text #4
      interceptions = i.find('td',{'data-stat':'interceptions'}).text #5
      crosses = i.find('td',{'data-stat':'crosses'}).text #6
      shots = i.find('td',{'data-stat':'shots_total'}).text
      shots_on_target = i.find('td',{'data-stat':'shots_on_target'}).text
      away_stats_list.append(list((number,fouls,assists,offside,interceptions,crosses,shots,shots_on_target)))
    
    
  else:
    temp = soup2.find('table',id='stats_'+away_code+'_summary').find('tbody')
    temp2 = soup2.find('table',id='stats_'+away_code+'_misc').find('tbody')
    for i,j in zip(temp,temp2):
      if i.find('td') == -1 or j.find('td') == -1:
         continue
      
      number = i.find('td',{'data-stat':'shirtnumber'}).text
      fouls = j.find('td',{'data-stat':'fouls'}).text     #2
      assists = i.find('td',{'data-stat':'assists'}).text #3
      offside = j.find('td',{'data-stat':'offsides'}).text #4
      interceptions = j.find('td',{'data-stat':'interceptions'}).text #5
      crosses = j.find('td',{'data-stat':'crosses'}).text #6
      shots = i.find('td',{'data-stat':'shots_total'}).text
      shots_on_target = i.find('td',{'data-stat':'shots_on_target'}).text
      away_stats_list.append(list((number,fouls,assists,offside,interceptions,crosses,shots,shots_on_target)))


  home_fouls_list = []
  home_assists_list= []
  home_offsides_list= []
  home_int_list= []
  home_crs_list= []
  home_shots_list= []
  home_sh_tar_list= []
  for i in range(len(home_stats_list)):
    home_fouls_list.append((home_stats_list[i][0],home_stats_list[i][1]))
    home_assists_list.append((home_stats_list[i][0],home_stats_list[i][2]))
    home_offsides_list.append((home_stats_list[i][0],home_stats_list[i][3])) 
    home_int_list.append((home_stats_list[i][0],home_stats_list[i][4]))
    home_crs_list.append((home_stats_list[i][0],home_stats_list[i][5]))  
    home_shots_list.append((home_stats_list[i][0],home_stats_list[i][6]))   
    home_sh_tar_list.append((home_stats_list[i][0],home_stats_list[i][7]))
 

  away_fouls_list = []
  away_assists_list= []
  away_offsides_list= []
  away_int_list= []
  away_crs_list= []
  away_shots_list= []
  away_sh_tar_list= []
  for i in range(len(away_stats_list)):
    away_fouls_list.append((away_stats_list[i][0],away_stats_list[i][1]))
    away_assists_list.append((away_stats_list[i][0],away_stats_list[i][2]))
    away_offsides_list.append((away_stats_list[i][0],away_stats_list[i][3])) 
    away_int_list.append((away_stats_list[i][0],away_stats_list[i][4]))
    away_crs_list.append((away_stats_list[i][0],away_stats_list[i][5]))  
    away_shots_list.append((away_stats_list[i][0],away_stats_list[i][6]))   
    away_sh_tar_list.append((away_stats_list[i][0],away_stats_list[i][7]))
 
  home_fouls_name = dict(home_fouls_list)
  home_assists_name = dict(home_assists_list)  
  home_offsides_name = dict(home_offsides_list)  
  home_int_name = dict(home_int_list)
  home_crs_name = dict(home_crs_list)
  home_shots_name = dict(home_shots_list)
  home_sh_tar_name = dict(home_sh_tar_list)

  away_fouls_name = dict(away_fouls_list)
  away_assists_name = dict(away_assists_list)  
  away_offsides_name = dict(away_offsides_list)  
  away_int_name = dict(away_int_list)
  away_crs_name = dict(away_crs_list)
  away_shots_name = dict(away_shots_list)
  away_sh_tar_name = dict(away_sh_tar_list)

#TEAM DATA
  url_stats = url + "statistics/?iv=true"
  page = requests.get(url_stats)
  if(page.status_code == 200):
    soup = bs.BeautifulSoup(page.content, "html.parser")

  try:
    total_attempts_home = soup.find("div", {"class":"total-attempts--value graph-bar--number-value graph-bar--number-value__home-team","data-bind":"text: _.findWhere(homeTeam.statistics,{type:'ATTEMPTS'}) ? _.findWhere(homeTeam.statistics,{type:'ATTEMPTS'}).value : 0"}).text
    total_attempts_away = soup.find("div", {"class":"total-attempts--value graph-bar--number-value graph-bar--number-value__away-team", "data-bind":"text: _.findWhere(awayTeam.statistics,{type:'ATTEMPTS'}) ? _.findWhere(awayTeam.statistics,{type:'ATTEMPTS'}).value : 0"}).text
  except:
    total_attempts_home = "Not Provided"
    total_attempts_away = "Not Provided"

  try:
    on_target_home = soup.find("div",{"class":"total-attempts--value graph-bar--number-value graph-bar--number-value__home-team", "data-bind":"text: homeAttempsOn"}).text
    on_target_away = soup.find("div",{"class":"total-attempts--value graph-bar--number-value graph-bar--number-value__away-team","data-bind":"text: awayAttempsOn"}).text
  except:
    on_target_home = "Not Provided"
    on_target_away = "Not Provided"


  try:
    off_target_home = soup.find("div",{"class":"total-attempts--value graph-bar--number-value graph-bar--number-value__home-team", "data-bind":"text: homeAttempsOff"}).text
    off_target_away = soup.find("div",{"class":"total-attempts--value graph-bar--number-value graph-bar--number-value__away-team","data-bind":"text: awayAttempsOff"}).text
  except:
    off_target_home = "Not Provided"
    off_target_away = "Not Provided"


  try:
    blocked_home = soup.find("div",{"class":"total-attempts--value graph-bar--number-value graph-bar--number-value__home-team", "data-bind":"text: _.findWhere(homeTeam.statistics,{type:'SHOTS_BLOCKED'}) ? _.findWhere(homeTeam.statistics,{type:'SHOTS_BLOCKED'}).value : 0"}).text
    blocked_away = soup.find("div",{"class":"total-attempts--value graph-bar--number-value graph-bar--number-value__away-team","data-bind":"text: _.findWhere(awayTeam.statistics,{type:'SHOTS_BLOCKED'}) ? _.findWhere(awayTeam.statistics,{type:'SHOTS_BLOCKED'}).value : 0"}).text
  except:
    blocked_home = "Not Provided"
    blocked_away = "Not Provided"

  try:
    woodwork_home = soup.find("div",{"class":"total-attempts--value graph-bar--number-value graph-bar--number-value__home-team", "data-bind":"text: _.findWhere(homeTeam.statistics,{type:'AGAINST_WOODWORK'}) ? _.findWhere(homeTeam.statistics,{type:'AGAINST_WOODWORK'}).value : 0"}).text
    woodwork_away = soup.find("div",{"class":"total-attempts--value graph-bar--number-value graph-bar--number-value__away-team","data-bind":"text: _.findWhere(awayTeam.statistics,{type:'AGAINST_WOODWORK'}) ? _.findWhere(awayTeam.statistics,{type:'AGAINST_WOODWORK'}).value : 0"}).text
  except:
    woodwork_home = "Not Provided"
    woodwork_away = "Not Provided"

  try:
    corners_home = soup.find("div",{"class":"corner--value graph-bar--number-value graph-bar--number-value__home-team", "data-bind":"text: homeCorner"}).text
    corners_away = soup.find("div",{"class":"corner--value graph-bar--number-value graph-bar--number-value__away-team","data-bind":"text: awayCorner"}).text
  except:
    corners_home = "Not Provided"
    corners_away = "Not Provided"

  try:
    offsides_home = soup.find("div",{"class":"offside--value graph-bar--number-value graph-bar--number-value__home-team", "data-bind":"text: homeOffside"}).text
    offsides_away = soup.find("div",{"class":"offside--value graph-bar--number-value graph-bar--number-value__away-team","data-bind":"text: awayOffside"}).text
  except:
    offsides_home = "Not Provided"
    offsides_away = "Not Provided"

  try:
    possession_home = soup.find("div",{"class":"ball-possession-value graph-circle-number-value graph-circle-number-value__home-team", "data-bind":"text: homeBallPossession + '%'"}).text
    possession_away = soup.find("div",{"class":"ball-possession-value graph-circle-number-value graph-circle-number-value__away-team","data-bind":"text: awayBallPossession + '%'"}).text
  except:
    possession_home = "Not Provided"
    possession_away = "Not Provided"

  try:
    passing_accuracy_home = soup.find("span", {"class":"label", "data-bind":"text: homePassesCompletion + '%'"}).text
    passing_accuracy_away = soup.find("span", {"class":"label", "data-bind":"text: awayPassesCompletion + '%'"}).text
  except:
    passing_accuracy_home = "Not Provided"
    passing_accuracy_away = "Not Provided"

  try:
    passes_home = soup.find("div",{"class":"passes--value graph-bar--number-value graph-bar--number-value__home-team", "data-bind":"text: _.findWhere(homeTeam.statistics,{type:'PASSES_ATTEMPTED'}) ? _.findWhere(homeTeam.statistics,{type:'PASSES_ATTEMPTED'}).value : 0"}).text
    passes_away = soup.find("div",{"class":"passes--value graph-bar--number-value graph-bar--number-value__away-team","data-bind":"text: _.findWhere(awayTeam.statistics,{type:'PASSES_ATTEMPTED'}) ? _.findWhere(awayTeam.statistics,{type:'PASSES_ATTEMPTED'}).value : 0"}).text
  except:
    passes_home = "Not Provided"
    passes_away = "Not Provided"

  try:
    passes_completed_home = soup.find("div",{"class":"passes-completed--value graph-bar--number-value graph-bar--number-value__home-team", "data-bind":"text: _.findWhere(homeTeam.statistics,{type:'PASSES_COMPLETED'}) ? _.findWhere(homeTeam.statistics,{type:'PASSES_COMPLETED'}).value : 0"}).text
    passes_completed_away = soup.find("div",{"class":"passes-completed--value graph-bar--number-value graph-bar--number-value__away-team","data-bind":"text: _.findWhere(awayTeam.statistics,{type:'PASSES_COMPLETED'}) ? _.findWhere(awayTeam.statistics,{type:'PASSES_COMPLETED'}).value : 0"}).text
  except:
    passes_completed_home = "Not Provided"
    passes_completed_away = "Not Provided"
  
  try:
    distance_covered_home = soup.find("div",{"class":"distance-run--value graph-bar--number-value graph-bar--number-value__home-team", "data-bind":"text: distance.toFixed(1) + ' km'"}).text
    distance_covered_away = soup.find("div",{"class":"distance-run--value graph-bar--number-value graph-bar--number-value__away-team","data-bind":"text: distance.toFixed(1) + ' km'"}).text
  except:
    distance_covered_home = "Not Provided"
    distance_covered_away = "Not Provided"
 
  try:
    balls_recovered_home = soup.find("div",{"class":"balls-recovered--value graph-bar--number-value graph-bar--number-value__home-team", "data-bind":"text: homeRecoveredBalls"}).text
    balls_recovered_away = soup.find("div",{"class":"balls-recovered--value graph-bar--number-value graph-bar--number-value__away-team","data-bind":"text: awayRecoveredBalls"}).text
  except:
    balls_recovered_home = "Not Provided"
    balls_recovered_away = "Not Provided"

  try:
    tackles_home = soup.find("div", {"class":"taclkles--value graph-bar--number-value graph-bar--number-value__home-team", "data-bind":"text: _.findWhere(homeTeam.statistics,{type:'TACKLES_COMPLETED'}) ? _.findWhere(homeTeam.statistics,{type:'TACKLES_COMPLETED'}).value : 0"}).text
    tackles_away = soup.find("div", {"class":"taclkles--value graph-bar--number-value graph-bar--number-value__away-team", "data-bind":"text: _.findWhere(awayTeam.statistics,{type:'TACKLES_COMPLETED'}) ? _.findWhere(awayTeam.statistics,{type:'TACKLES_COMPLETED'}).value : 0"}).text
  except:
    tackles_home = "Not Provided"
    tackles_away = "Not Provided"

  try:
    blocks_home = soup.find("div", {"class":"blocks-completed--value graph-bar--number-value graph-bar--number-value__home-team", "data-bind":"text: _.findWhere(homeTeam.statistics,{type:'BLOCKED'}) ? _.findWhere(homeTeam.statistics,{type:'BLOCKED'}).value : 0"}).text
    blocks_away = soup.find("div", {"class":"blocks-completed--value graph-bar--number-value graph-bar--number-value__away-team", "data-bind":"text: _.findWhere(awayTeam.statistics,{type:'BLOCKED'}) ? _.findWhere(awayTeam.statistics,{type:'BLOCKED'}).value : 0"}).text
  except:
    blocks_home = "Not Provided"
    blocks_away = "Not Provided"

  try:
    clearances_home = soup.find("div",{"class":"clearances-completed--value graph-bar--number-value graph-bar--number-value__home-team", "data-bind":"text: _.findWhere(homeTeam.statistics,{type:'CLEARANCES_COMPLETED'}) ? _.findWhere(homeTeam.statistics,{type:'CLEARANCES_COMPLETED'}).value : 0"}).text
    clearances_away = soup.find("div",{"class":"clearances-completed--value graph-bar--number-value graph-bar--number-value__away-team","data-bind":"text: _.findWhere(awayTeam.statistics,{type:'CLEARANCES_COMPLETED'}) ? _.findWhere(awayTeam.statistics,{type:'CLEARANCES_COMPLETED'}).value : 0"}).text
  except:
    clearances_home = "Not Provided"
    clearances_away = "Not Provided"

  try:
    yellow_cards_home = soup.find("div",{"class":"yellow-cards--value graph-bar--number-value graph-bar--number-value__home-team", "data-bind":"text: homeYellowCards"}).text
    yellow_cards_away = soup.find("div",{"class":"yellow-cards--value graph-bar--number-value graph-bar--number-value__away-team","data-bind":"text: awayYellowCards"}).text
  except:
    yellow_cards_home = "Not Provided"
    yellow_cards_away = "Not Provided"

  try:
    red_cards_home = soup.find("div",{"class":"red-cards--value graph-bar--number-value graph-bar--number-value__home-team", "data-bind":"text: homeRedCards"}).text
    red_cards_away = soup.find("div",{"class":"red-cards--value graph-bar--number-value graph-bar--number-value__away-team","data-bind":"text: awayRedCards"}).text
  except:
    red_cards_home = "Not Provided"
    red_cards_away = "Not Provided"

  try:
    fouls_home = soup.find("div",{"class":"fouls-committed--value graph-bar--number-value graph-bar--number-value__home-team", "data-bind":"text: _.findWhere(homeTeam.statistics,{type:'FOULS_COMMITTED'}) ? _.findWhere(homeTeam.statistics,{type:'FOULS_COMMITTED'}).value : 0"}).text
    fouls_away = soup.find("div",{"class":"fouls-committed--value graph-bar--number-value graph-bar--number-value__away-team","data-bind":"text: _.findWhere(awayTeam.statistics,{type:'FOULS_COMMITTED'}) ? _.findWhere(awayTeam.statistics,{type:'FOULS_COMMITTED'}).value : 0"}).text
  except:
    fouls_home = "Not Provided"
    fouls_away = "Not Provided"

  import pandas as pd

  all_players_list_bruh = []
  all_players_list_bruh.extend(home_players_list)
  all_players_list_bruh.append(coach)
  all_players_list_bruh.extend(away_players_list)
  all_players_list_bruh.append(coach_away)
  all_players_list.extend(all_players_list_bruh)
  teams = []
  for i in all_players_list_bruh:
      if i in home_players_list:
          teams.append(team_home_name)
      elif i == coach:
          s = team_home_name + "(COACH)"
          teams.append(s)
      elif i in away_players_list:
          teams.append(team_away_name)
      elif i == coach_away:
          s = team_away_name + "(COACH)"
          teams.append(s)

  joined_game_home_name = []
  joined_game_home_time = []
  for i,j in joined_game_home:
      joined_game_home_name.append(i)
      joined_game_home_time.append(j)

  joined_game_away_name = []
  joined_game_away_time = []
  for i,j in joined_game_away:
      joined_game_away_name.append(i)
      joined_game_away_time.append(j)

  did_not_finish_home_name = []
  did_not_finish_home_time = []
  for i,j in did_not_finish_home:
      did_not_finish_home_name.append(i)
      did_not_finish_home_time.append(j)


  did_not_finish_away_name = []
  did_not_finish_away_time = []
  for i,j in did_not_finish_away:
      did_not_finish_away_name.append(i)
      did_not_finish_away_time.append(j)

  joined_game_home_time

#   started_all_pd = []
  for i in home_players_list:
      if i in started_home:
          started_all_pd.append("0'")
      else:
          if(i in joined_game_home_name):
              started_all_pd.append(joined_game_home_time[joined_game_home_name.index(i)])
          else:
              started_all_pd.append("Did not play")
  started_all_pd.append("Not applicable")
  for i in away_players_list:
      if i in started_away:
          started_all_pd.append("0'")
      else:
          if(i in joined_game_away_name):
              started_all_pd.append(joined_game_away_time[joined_game_away_name.index(i)])
          else:
              started_all_pd.append("Did not play")
  started_all_pd.append("Not applicable")

#   finished_all = []
  for i in home_players_list:
      if i not in did_not_finish_home_name and (i in started_home or i in joined_game_home_name):
          finished_all.append("Finished")
      elif i in did_not_finish_home_name:
          s = "Substituted at " + did_not_finish_home_time[did_not_finish_home_name.index(i)]
          finished_all.append(s)

      else:
          finished_all.append("Not applicable")
  finished_all.append("----------")
  for i in away_players_list:
      if i not in did_not_finish_away_name and (i in started_away or i in joined_game_away_name):
          finished_all.append("Finished")
      elif i in did_not_finish_away_name:
          s = "Substituted at " + did_not_finish_away_time[did_not_finish_away_name.index(i)]
          finished_all.append(s)

      else:
          finished_all.append("Not applicable")

  finished_all.append("--------")

#   goals = []
  for i in home_players_list:
      if i in home_scorer_name:
          s = ""
          for j in home_scorer_name[i]:
              s += " " + j
          strs = "Scored at " + s
          goals.append(strs)
      else:
          goals.append("Not scored")

  goals.append("---------")
  for i in away_players_list:
      if i in away_scorer_name:
          s = ""
          for j in away_scorer_name[i]:
              s += " " + j
          strs = "Scored at " + s
          goals.append(strs)
      else:
          goals.append("Not scored")
  goals.append("----------")


  for i in home_players_numbers:
    if i in home_fouls_name:
        fouls_player.append(home_fouls_name[i])
    else:
        fouls_player.append("Not applicable")

    if i in home_assists_name:
        assists_player.append(home_assists_name[i])
    else:
        assists_player.append("Not applicable")

    if i in home_offsides_name:
        offside_player.append(home_offsides_name[i])
    else:
        offside_player.append("Not applicable")

    if i in home_int_name:
        inter_player.append(home_int_name[i])
    else:
        inter_player.append("Not applicable")

    if i in home_crs_name:
        crs_player.append(home_crs_name[i])
    else:
        crs_player.append("Not applicable")

    if i in home_shots_name:
        shots_player.append(home_shots_name[i])
    else:
        shots_player.append("Not applicable")

    if i in home_sh_tar_name:
        sh_tar_player.append(home_sh_tar_name[i])
    else:
        sh_tar_player.append("Not applicable")

  fouls_player.append("---------")
  assists_player.append("---------")
  offside_player.append("---------")
  inter_player.append("---------")
  crs_player.append("---------")
  shots_player.append("---------")
  sh_tar_player.append("---------")

  for i in away_players_numbers:
    if i in away_fouls_name:
        fouls_player.append(away_fouls_name[i])
    else:
        fouls_player.append("Not applicable")

    if i in away_assists_name:
        assists_player.append(away_assists_name[i])
    else:
        assists_player.append("Not applicable")

    if i in away_offsides_name:
        offside_player.append(away_offsides_name[i])
    else:
        offside_player.append("Not applicable")

    if i in away_int_name:
        inter_player.append(away_int_name[i])
    else:
        inter_player.append("Not applicable")

    if i in away_crs_name:
        crs_player.append(away_crs_name[i])
    else:
        crs_player.append("Not applicable")

    if i in away_shots_name:
        shots_player.append(away_shots_name[i])
    else:
        shots_player.append("Not applicable")

    if i in away_sh_tar_name:
        sh_tar_player.append(away_sh_tar_name[i])
    else:
        sh_tar_player.append("Not applicable")

  fouls_player.append("---------")
  assists_player.append("---------")
  offside_player.append("---------")
  inter_player.append("---------")
  crs_player.append("---------")
  shots_player.append("---------")
  sh_tar_player.append("---------")


  teams_list.extend(teams)
  team_names.extend([team_home_name, team_away_name])
  locations.extend([location, location])
  team_scores.extend([team_home_score, team_away_score])

  total_attempts.extend([total_attempts_home, total_attempts_away])

  on_targets.extend([on_target_home, on_target_away])

  off_targets.extend([off_target_home, off_target_away])

  blocked.extend([blocked_home, blocked_away])

  woodworks.extend([woodwork_home, woodwork_away])

  corners.extend([corners_home, corners_away])

  offsides.extend([offsides_home, offsides_away])

  possession.extend([possession_home, possession_away])

  passes.extend([passes_home, passes_away])

  passing_accuracy.extend([passing_accuracy_home, passing_accuracy_away])

  passes_completed.extend([passes_completed_home, passes_completed_away])

  distance_covered.extend([distance_covered_home, distance_covered_away])

  balls_recovered.extend([balls_recovered_home, balls_recovered_away])

  tackles.extend([tackles_home, tackles_away])

  clearances.extend([clearances_home, clearances_away])
  blocks.extend([blocks_home, blocks_away])

  yellow_cards.extend([yellow_cards_home, yellow_cards_away])

  red_cards.extend([red_cards_home, red_cards_away])

  fouls_list.extend([fouls_home, fouls_away])
  



#Write a similar function. but this time to obtain results for other years

This step is necessary because the website format is different in the year 2019 for the official website. Hence the scraping parameters and rules will be different. Therefore we have to use different functions to scrape the years before 2019.

In [0]:
def get_dataset(url,url2):
  page = requests.get(url)
  if(page.status_code == 200):
      soup = bs.BeautifulSoup(page.content, "html.parser")
  page2 = requests.get(url2)
  if(page2.status_code == 200):
      soup2 = bs.BeautifulSoup(page2.content, "html.parser")

  text = getText(page)

  



  team_home_score = soup.find('span', {'class' : 'js-team--home-score'}).text
  

  team_away_score = soup.find('span', {'class' : 'js-team--away-score'}).text
  
  try:
    penal_score = soup.find('div', {'class' : 'js-match-status-aggr match-status-aggr'}).text
    temp = penal_score.split()
    home_penal = int(temp[1][0])
    away_penal = int(temp[1][-1])
  except:
    home_penal = 0
    away_penal = 0

  try:
      team_home_name = soup.find('div', {'class' : 'team-home is-club match--winner'}).find('div', {'class' : 'team-name'}).find("div").find('span', {'class' : 'js-fitty'}).text
  except:
      team_home_name = soup.find('div', {'class' : 'team-home is-club '}).find('div', {'class' : 'team-name'}).find("div").find('span', {'class' : 'js-fitty'}).text
  team_home_name = team_home_name.strip()

  try:
      team_away_name = soup.find('div', {'class' : 'team-away is-club match--winner'}).find('div', {'class' : 'team-name'}).find("div").find('span', {'class' : 'js-fitty'}).text
  except:
      team_away_name = soup.find('div', {'class' : 'team-away is-club '}).find('div', {'class' : 'team-name'}).find("div").find('span', {'class' : 'js-fitty'}).text
  team_away_name = team_away_name.strip()

  

  code_links = soup2.find_all('img',{'class':'teamlogo'})[:2]
  home_code = code_links[0]['src'].split('/')[-1].replace('.png','')
  away_code = code_links[1]['src'].split('/')[-1].replace('.png','')
  year = int(url2.split('-')[-3])

  home_players = soup.find_all("li", {"class":"squad--team-player squad--team-home squad--team-home-player"})

  home_players_list = []
  for i in home_players:
      a = i.find("span", {"class": "squad--player-name"}).find("div").find('span', {'class' : 'js-fitty'}).text
      home_players_list.append(a.strip())

  home_players_list = list(dict.fromkeys(home_players_list))
  

  if url == "https://www.uefa.com//uefachampionsleague/match/2019616--sevilla-vs-leicester/" :
    coach = "Jorge Sampaoli"
  else:
    coach = home_players_list[-1]
    home_players_list.pop()
  

  home_players_numbers = []
  for i in home_players:
    a = i.find('span',{'class':'squad--player-num'})
    if a == None:
      continue
    else:
      home_players_numbers.append(a.text.strip())
  home_players_numbers = list(dict.fromkeys(home_players_numbers))

  away_players = soup.find_all("li", {"class":"squad--team-player squad--team-away squad--team-away-player"})

  away_players_list = []
  for i in away_players:
      a = i.find("span", {"class": "squad--player-name"}).find("div").find('span', {'class' : 'js-fitty'}).text
      away_players_list.append(a.strip())

  away_players_list = list(dict.fromkeys(away_players_list))
  
  coach_away = away_players_list[-1]
  away_players_list.pop()

  away_players_numbers = []
  for i in away_players:
    a = i.find('span',{'class':'squad--player-num'})
    if a == None:
      continue
    else:
      away_players_numbers.append(a.text.strip())
  away_players_numbers = list(dict.fromkeys(away_players_numbers))

  home_stats_list = []
  if year <= 2018:
    temp = soup2.find('table',id='stats_'+home_code+'_summary').find('tbody')
    for i in temp:
      if i.find('td') == -1:
        continue
      number = i.find('td',{'data-stat':'shirtnumber'}).text
      fouls = i.find('td',{'data-stat':'fouls'}).text     #2
      assists = i.find('td',{'data-stat':'assists'}).text #3
      offside = i.find('td',{'data-stat':'offsides'}).text #4
      interceptions = i.find('td',{'data-stat':'interceptions'}).text #5
      crosses = i.find('td',{'data-stat':'crosses'}).text #6
      shots = i.find('td',{'data-stat':'shots_total'}).text
      shots_on_target = i.find('td',{'data-stat':'shots_on_target'}).text
      home_stats_list.append(list((number,fouls,assists,offside,interceptions,crosses,shots,shots_on_target)))
    
    

  else:
    temp = soup2.find('table',id='stats_'+home_code+'_summary').find('tbody')
    temp2 = soup2.find('table',id='stats_'+home_code+'_misc').find('tbody')
    for i,j in zip(temp,temp2):
      if i.find('td') == -1 or j.find('td') == -1:
        continue
      
      number = i.find('td',{'data-stat':'shirtnumber'}).text
      fouls = j.find('td',{'data-stat':'fouls'}).text     #2
      assists = i.find('td',{'data-stat':'assists'}).text #3
      offside = j.find('td',{'data-stat':'offsides'}).text #4
      interceptions = j.find('td',{'data-stat':'interceptions'}).text #5
      crosses = j.find('td',{'data-stat':'crosses'}).text #6
      shots = i.find('td',{'data-stat':'shots_total'}).text
      shots_on_target = i.find('td',{'data-stat':'shots_on_target'}).text
      home_stats_list.append(list((number,fouls,assists,offside,interceptions,crosses,shots,shots_on_target)))
    
  
  
  away_stats_list = []
  if year <=2018:
    temp = soup2.find('table',id='stats_'+away_code+'_summary').find('tbody')
    for i in temp:
      if i.find('td') == -1:
        continue
      
      number = i.find('td',{'data-stat':'shirtnumber'}).text
      fouls = i.find('td',{'data-stat':'fouls'}).text     #2
      assists = i.find('td',{'data-stat':'assists'}).text #3
      offside = i.find('td',{'data-stat':'offsides'}).text #4
      interceptions = i.find('td',{'data-stat':'interceptions'}).text #5
      crosses = i.find('td',{'data-stat':'crosses'}).text #6
      shots = i.find('td',{'data-stat':'shots_total'}).text
      shots_on_target = i.find('td',{'data-stat':'shots_on_target'}).text
      away_stats_list.append(list((number,fouls,assists,offside,interceptions,crosses,shots,shots_on_target)))
    
    
  else:
    temp = soup2.find('table',id='stats_'+away_code+'_summary').find('tbody')
    temp2 = soup2.find('table',id='stats_'+away_code+'_misc').find('tbody')
    for i,j in zip(temp,temp2):
      if i.find('td') == -1 or j.find('td') == -1:
         continue
      
      number = i.find('td',{'data-stat':'shirtnumber'}).text
      fouls = j.find('td',{'data-stat':'fouls'}).text     #2
      assists = i.find('td',{'data-stat':'assists'}).text #3
      offside = j.find('td',{'data-stat':'offsides'}).text #4
      interceptions = j.find('td',{'data-stat':'interceptions'}).text #5
      crosses = j.find('td',{'data-stat':'crosses'}).text #6
      shots = i.find('td',{'data-stat':'shots_total'}).text
      shots_on_target = i.find('td',{'data-stat':'shots_on_target'}).text
      away_stats_list.append(list((number,fouls,assists,offside,interceptions,crosses,shots,shots_on_target)))
    
   

  home_scorers_list = []
  temp = soup.find("ul", {'class':'match--scorers js-home-team--scorers'})
  scorers = temp.find_all("li", {'class':'scorer'})

  for i in scorers:
      home_scorers_list.append(i.text.strip())

  away_scorers_list = []
  temp = soup.find("ul", {'class':'match--scorers js-away-team--scorers'})
  scorers = temp.find_all("li", {'class':'scorer'})

  for i in scorers:
      away_scorers_list.append(i.text.strip())


  count = 0
  started_home = []
  bench_home = []
  for i in home_players_list:
      if("(GK)" in i):
          count+=1
      if(count < 2):
          started_home.append(i)
      else:
          bench_home.append(i)

  count = 0
  started_away = []
  bench_away = []
  for i in away_players_list:
      if("(GK)" in i):
          count+=1
      if(count < 2):
          started_away.append(i)
      else:
          bench_away.append(i)

  home_fouls_list = []
  home_assists_list= []
  home_offsides_list= []
  home_int_list= []
  home_crs_list= []
  home_shots_list= []
  home_sh_tar_list= []
  for i in range(len(home_stats_list)):
    home_fouls_list.append((home_stats_list[i][0],home_stats_list[i][1]))
    home_assists_list.append((home_stats_list[i][0],home_stats_list[i][2]))
    home_offsides_list.append((home_stats_list[i][0],home_stats_list[i][3])) 
    home_int_list.append((home_stats_list[i][0],home_stats_list[i][4]))
    home_crs_list.append((home_stats_list[i][0],home_stats_list[i][5]))  
    home_shots_list.append((home_stats_list[i][0],home_stats_list[i][6]))   
    home_sh_tar_list.append((home_stats_list[i][0],home_stats_list[i][7]))
 

  away_fouls_list = []
  away_assists_list= []
  away_offsides_list= []
  away_int_list= []
  away_crs_list= []
  away_shots_list= []
  away_sh_tar_list= []
  for i in range(len(away_stats_list)):
    away_fouls_list.append((away_stats_list[i][0],away_stats_list[i][1]))
    away_assists_list.append((away_stats_list[i][0],away_stats_list[i][2]))
    away_offsides_list.append((away_stats_list[i][0],away_stats_list[i][3])) 
    away_int_list.append((away_stats_list[i][0],away_stats_list[i][4]))
    away_crs_list.append((away_stats_list[i][0],away_stats_list[i][5]))  
    away_shots_list.append((away_stats_list[i][0],away_stats_list[i][6]))   
    away_sh_tar_list.append((away_stats_list[i][0],away_stats_list[i][7]))
 
  home_fouls_name = dict(home_fouls_list)
  home_assists_name = dict(home_assists_list)  
  home_offsides_name = dict(home_offsides_list)  
  home_int_name = dict(home_int_list)
  home_crs_name = dict(home_crs_list)
  home_shots_name = dict(home_shots_list)
  home_sh_tar_name = dict(home_sh_tar_list)

  away_fouls_name = dict(away_fouls_list)
  away_assists_name = dict(away_assists_list)  
  away_offsides_name = dict(away_offsides_list)  
  away_int_name = dict(away_int_list)
  away_crs_name = dict(away_crs_list)
  away_shots_name = dict(away_shots_list)
  away_sh_tar_name = dict(away_sh_tar_list)

  substitutions_home = []
  for i in home_players:
      name = i.find("span", {"class": "squad--player-name"}).find("div").find('span', {'class' : 'js-fitty'}).text
      name=" ".join(name.split())
      
      a = i.find("span", {"class":"lineups--events"})
      if(a==None):
          continue
      b = a.find_all("img", alt=True)
      for img in b:
          if(img["alt"] == "Substitution"):
              g = img.findNext("span").text
              substitutions_home.append((name,g))

  substitutions_home = list(dict.fromkeys(substitutions_home))
  

  substitutions_away = []
  for i in away_players:
      name = i.find("span", {"class": "squad--player-name"}).find("div").find('span', {'class' : 'js-fitty'}).text
      name=" ".join(name.split())
      
      a = i.find("span", {"class":"lineups--events"})
      if(a==None):
          continue
      b = a.find_all("img", alt=True)
      for img in b:
          if(img["alt"] == "Substitution"):
              g = img.findNext("span").text
              substitutions_away.append((name,g))

  substitutions_away = list(dict.fromkeys(substitutions_away))
  

  l = len(substitutions_home)
  l = l//2
  did_not_finish_home = substitutions_home[:l]
  

  joined_game_home = substitutions_home[l:]
  

  l = len(substitutions_away)
  l = l//2
  did_not_finish_away = substitutions_away[:l]
  

  joined_game_away = substitutions_away[l:]
  

  home_scorer_name ={}
  for i in home_scorers_list:
      h = i.split()
      name = ""
      goal_index = 0
      for s in h:
          if(s[0].isdigit()):
              break
          goal_index +=1
          name+= s + " "
      name = name.strip()
      home_scorer_name[name] = []
      home_scorer_name[name] = h[goal_index:]

  away_scorer_name ={}
  for i in away_scorers_list:
      h = i.split()
      name = ""
      goal_index = 0
      for s in h:
          if(s[0].isdigit()):
              break
          goal_index +=1
          name+= s + " "
      name = name.strip()
      away_scorer_name[name] = []
      away_scorer_name[name] = h[goal_index:]

  
  stadium_name = soup.find("span", {"class":"match-location_stadium"}).text
  stadium_city = soup.find("span", {"class":"match-location_venue"}).text
  location = stadium_name + ", " + stadium_city
  
  try:
    total_attempts_home = soup.find("div", {"class":"total-attempts--value graph-bar--number-value graph-bar--number-value__home-team","data-bind":"text: _.findWhere(homeTeam.statistics,{type:'ATTEMPTS'}) ? _.findWhere(homeTeam.statistics,{type:'ATTEMPTS'}).value : 0"}).text
    total_attempts_away = soup.find("div", {"class":"total-attempts--value graph-bar--number-value graph-bar--number-value__away-team", "data-bind":"text: _.findWhere(awayTeam.statistics,{type:'ATTEMPTS'}) ? _.findWhere(awayTeam.statistics,{type:'ATTEMPTS'}).value : 0"}).text
  except:
    total_attempts_home = "Not Provided"
    total_attempts_away = "Not Provided"

  try:
    on_target_home = soup.find("div",{"class":"total-attempts--value graph-bar--number-value graph-bar--number-value__home-team", "data-bind":"text: homeAttempsOn"}).text
    on_target_away = soup.find("div",{"class":"total-attempts--value graph-bar--number-value graph-bar--number-value__away-team","data-bind":"text: awayAttempsOn"}).text
  except:
    on_target_home = "Not Provided"
    on_target_away = "Not Provided"


  try:
    off_target_home = soup.find("div",{"class":"total-attempts--value graph-bar--number-value graph-bar--number-value__home-team", "data-bind":"text: homeAttempsOff"}).text
    off_target_away = soup.find("div",{"class":"total-attempts--value graph-bar--number-value graph-bar--number-value__away-team","data-bind":"text: awayAttempsOff"}).text
  except:
    off_target_home = "Not Provided"
    off_target_away = "Not Provided"


  try:
    blocked_home = soup.find("div",{"class":"total-attempts--value graph-bar--number-value graph-bar--number-value__home-team", "data-bind":"text: _.findWhere(homeTeam.statistics,{type:'SHOTS_BLOCKED'}) ? _.findWhere(homeTeam.statistics,{type:'SHOTS_BLOCKED'}).value : 0"}).text
    blocked_away = soup.find("div",{"class":"total-attempts--value graph-bar--number-value graph-bar--number-value__away-team","data-bind":"text: _.findWhere(awayTeam.statistics,{type:'SHOTS_BLOCKED'}) ? _.findWhere(awayTeam.statistics,{type:'SHOTS_BLOCKED'}).value : 0"}).text
  except:
    blocked_home = "Not Provided"
    blocked_away = "Not Provided"

  try:
    woodwork_home = soup.find("div",{"class":"total-attempts--value graph-bar--number-value graph-bar--number-value__home-team", "data-bind":"text: _.findWhere(homeTeam.statistics,{type:'AGAINST_WOODWORK'}) ? _.findWhere(homeTeam.statistics,{type:'AGAINST_WOODWORK'}).value : 0"}).text
    woodwork_away = soup.find("div",{"class":"total-attempts--value graph-bar--number-value graph-bar--number-value__away-team","data-bind":"text: _.findWhere(awayTeam.statistics,{type:'AGAINST_WOODWORK'}) ? _.findWhere(awayTeam.statistics,{type:'AGAINST_WOODWORK'}).value : 0"}).text
  except:
    woodwork_home = "Not Provided"
    woodwork_away = "Not Provided"

  try:
    corners_home = soup.find("div",{"class":"corner--value graph-bar--number-value graph-bar--number-value__home-team", "data-bind":"text: homeCorner"}).text
    corners_away = soup.find("div",{"class":"corner--value graph-bar--number-value graph-bar--number-value__away-team","data-bind":"text: awayCorner"}).text
  except:
    corners_home = "Not Provided"
    corners_away = "Not Provided"

  try:
    offsides_home = soup.find("div",{"class":"offside--value graph-bar--number-value graph-bar--number-value__home-team", "data-bind":"text: homeOffside"}).text
    offsides_away = soup.find("div",{"class":"offside--value graph-bar--number-value graph-bar--number-value__away-team","data-bind":"text: awayOffside"}).text
  except:
    offsides_home = "Not Provided"
    offsides_away = "Not Provided"

  try:
    possession_home = soup.find("div",{"class":"ball-possession-value graph-circle-number-value graph-circle-number-value__home-team", "data-bind":"text: homeBallPossession + '%'"}).text
    possession_away = soup.find("div",{"class":"ball-possession-value graph-circle-number-value graph-circle-number-value__away-team","data-bind":"text: awayBallPossession + '%'"}).text
  except:
    possession_home = "Not Provided"
    possession_away = "Not Provided"

  try:
    passing_accuracy_home = soup.find("span", {"class":"label", "data-bind":"text: homePassesCompletion + '%'"}).text
    passing_accuracy_away = soup.find("span", {"class":"label", "data-bind":"text: awayPassesCompletion + '%'"}).text
  except:
    passing_accuracy_home = "Not Provided"
    passing_accuracy_away = "Not Provided"

  try:
    passes_home = soup.find("div",{"class":"passes--value graph-bar--number-value graph-bar--number-value__home-team", "data-bind":"text: _.findWhere(homeTeam.statistics,{type:'PASSES_ATTEMPTED'}) ? _.findWhere(homeTeam.statistics,{type:'PASSES_ATTEMPTED'}).value : 0"}).text
    passes_away = soup.find("div",{"class":"passes--value graph-bar--number-value graph-bar--number-value__away-team","data-bind":"text: _.findWhere(awayTeam.statistics,{type:'PASSES_ATTEMPTED'}) ? _.findWhere(awayTeam.statistics,{type:'PASSES_ATTEMPTED'}).value : 0"}).text
  except:
    passes_home = "Not Provided"
    passes_away = "Not Provided"

  try:
    passes_completed_home = soup.find("div",{"class":"passes-completed--value graph-bar--number-value graph-bar--number-value__home-team", "data-bind":"text: _.findWhere(homeTeam.statistics,{type:'PASSES_COMPLETED'}) ? _.findWhere(homeTeam.statistics,{type:'PASSES_COMPLETED'}).value : 0"}).text
    passes_completed_away = soup.find("div",{"class":"passes-completed--value graph-bar--number-value graph-bar--number-value__away-team","data-bind":"text: _.findWhere(awayTeam.statistics,{type:'PASSES_COMPLETED'}) ? _.findWhere(awayTeam.statistics,{type:'PASSES_COMPLETED'}).value : 0"}).text
  except:
    passes_completed_home = "Not Provided"
    passes_completed_away = "Not Provided"
  
  try:
    distance_covered_home = soup.find("div",{"class":"distance-run--value graph-bar--number-value graph-bar--number-value__home-team", "data-bind":"text: distance.toFixed(1) + ' km'"}).text
    distance_covered_away = soup.find("div",{"class":"distance-run--value graph-bar--number-value graph-bar--number-value__away-team","data-bind":"text: distance.toFixed(1) + ' km'"}).text
  except:
    distance_covered_home = "Not Provided"
    distance_covered_away = "Not Provided"
 
  try:
    balls_recovered_home = soup.find("div",{"class":"balls-recovered--value graph-bar--number-value graph-bar--number-value__home-team", "data-bind":"text: homeRecoveredBalls"}).text
    balls_recovered_away = soup.find("div",{"class":"balls-recovered--value graph-bar--number-value graph-bar--number-value__away-team","data-bind":"text: awayRecoveredBalls"}).text
  except:
    balls_recovered_home = "Not Provided"
    balls_recovered_away = "Not Provided"

  try:
    tackles_home = soup.find("div", {"class":"taclkles--value graph-bar--number-value graph-bar--number-value__home-team", "data-bind":"text: _.findWhere(homeTeam.statistics,{type:'TACKLES_COMPLETED'}) ? _.findWhere(homeTeam.statistics,{type:'TACKLES_COMPLETED'}).value : 0"}).text
    tackles_away = soup.find("div", {"class":"taclkles--value graph-bar--number-value graph-bar--number-value__away-team", "data-bind":"text: _.findWhere(awayTeam.statistics,{type:'TACKLES_COMPLETED'}) ? _.findWhere(awayTeam.statistics,{type:'TACKLES_COMPLETED'}).value : 0"}).text
  except:
    tackles_home = "Not Provided"
    tackles_away = "Not Provided"

  try:
    blocks_home = soup.find("div", {"class":"blocks-completed--value graph-bar--number-value graph-bar--number-value__home-team", "data-bind":"text: _.findWhere(homeTeam.statistics,{type:'BLOCKED'}) ? _.findWhere(homeTeam.statistics,{type:'BLOCKED'}).value : 0"}).text
    blocks_away = soup.find("div", {"class":"blocks-completed--value graph-bar--number-value graph-bar--number-value__away-team", "data-bind":"text: _.findWhere(awayTeam.statistics,{type:'BLOCKED'}) ? _.findWhere(awayTeam.statistics,{type:'BLOCKED'}).value : 0"}).text
  except:
    blocks_home = "Not Provided"
    blocks_away = "Not Provided"

  try:
    clearances_home = soup.find("div",{"class":"clearances-completed--value graph-bar--number-value graph-bar--number-value__home-team", "data-bind":"text: _.findWhere(homeTeam.statistics,{type:'CLEARANCES_COMPLETED'}) ? _.findWhere(homeTeam.statistics,{type:'CLEARANCES_COMPLETED'}).value : 0"}).text
    clearances_away = soup.find("div",{"class":"clearances-completed--value graph-bar--number-value graph-bar--number-value__away-team","data-bind":"text: _.findWhere(awayTeam.statistics,{type:'CLEARANCES_COMPLETED'}) ? _.findWhere(awayTeam.statistics,{type:'CLEARANCES_COMPLETED'}).value : 0"}).text
  except:
    clearances_home = "Not Provided"
    clearances_away = "Not Provided"

  try:
    yellow_cards_home = soup.find("div",{"class":"yellow-cards--value graph-bar--number-value graph-bar--number-value__home-team", "data-bind":"text: homeYellowCards"}).text
    yellow_cards_away = soup.find("div",{"class":"yellow-cards--value graph-bar--number-value graph-bar--number-value__away-team","data-bind":"text: awayYellowCards"}).text
  except:
    yellow_cards_home = "Not Provided"
    yellow_cards_away = "Not Provided"

  try:
    red_cards_home = soup.find("div",{"class":"red-cards--value graph-bar--number-value graph-bar--number-value__home-team", "data-bind":"text: homeRedCards"}).text
    red_cards_away = soup.find("div",{"class":"red-cards--value graph-bar--number-value graph-bar--number-value__away-team","data-bind":"text: awayRedCards"}).text
  except:
    red_cards_home = "Not Provided"
    red_cards_away = "Not Provided"

  try:
    fouls_home = soup.find("div",{"class":"fouls-committed--value graph-bar--number-value graph-bar--number-value__home-team", "data-bind":"text: _.findWhere(homeTeam.statistics,{type:'FOULS_COMMITTED'}) ? _.findWhere(homeTeam.statistics,{type:'FOULS_COMMITTED'}).value : 0"}).text
    fouls_away = soup.find("div",{"class":"fouls-committed--value graph-bar--number-value graph-bar--number-value__away-team","data-bind":"text: _.findWhere(awayTeam.statistics,{type:'FOULS_COMMITTED'}) ? _.findWhere(awayTeam.statistics,{type:'FOULS_COMMITTED'}).value : 0"}).text
  except:
    fouls_home = "Not Provided"
    fouls_away = "Not Provided"

  import pandas as pd

  all_players_list_bruh = []
  all_players_list_bruh.extend(home_players_list)
  all_players_list_bruh.append(coach)
  all_players_list_bruh.extend(away_players_list)
  all_players_list_bruh.append(coach_away)
  all_players_list.extend(all_players_list_bruh)
  teams = []
  for i in all_players_list_bruh:
      if i in home_players_list:
          teams.append(team_home_name)
      elif i == coach:
          s = team_home_name + "(COACH)"
          teams.append(s)
      elif i in away_players_list:
          teams.append(team_away_name)
      elif i == coach_away:
          s = team_away_name + "(COACH)"
          teams.append(s)

  joined_game_home_name = []
  joined_game_home_time = []
  for i,j in joined_game_home:
      joined_game_home_name.append(i)
      joined_game_home_time.append(j)

  joined_game_away_name = []
  joined_game_away_time = []
  for i,j in joined_game_away:
      joined_game_away_name.append(i)
      joined_game_away_time.append(j)

  did_not_finish_home_name = []
  did_not_finish_home_time = []
  for i,j in did_not_finish_home:
      did_not_finish_home_name.append(i)
      did_not_finish_home_time.append(j)


  did_not_finish_away_name = []
  did_not_finish_away_time = []
  for i,j in did_not_finish_away:
      did_not_finish_away_name.append(i)
      did_not_finish_away_time.append(j)

  joined_game_home_time

#   started_all_pd = []
  for i in home_players_list:
      if i in started_home:
          started_all_pd.append("0'")
      else:
          if(i in joined_game_home_name):
              started_all_pd.append(joined_game_home_time[joined_game_home_name.index(i)])
          else:
              started_all_pd.append("Did not play")
  started_all_pd.append("Not applicable")
  for i in away_players_list:
      if i in started_away:
          started_all_pd.append("0'")
      else:
          if(i in joined_game_away_name):
              started_all_pd.append(joined_game_away_time[joined_game_away_name.index(i)])
          else:
              started_all_pd.append("Did not play")
  started_all_pd.append("Not applicable")

#   finished_all = []
  for i in home_players_list:
      if i not in did_not_finish_home_name and (i in started_home or i in joined_game_home_name):
          finished_all.append("Finished")
      elif i in did_not_finish_home_name:
          s = "Substituted at " + did_not_finish_home_time[did_not_finish_home_name.index(i)]
          finished_all.append(s)

      else:
          finished_all.append("Not applicable")
  finished_all.append("----------")
  for i in away_players_list:
      if i not in did_not_finish_away_name and (i in started_away or i in joined_game_away_name):
          finished_all.append("Finished")
      elif i in did_not_finish_away_name:
          s = "Substituted at " + did_not_finish_away_time[did_not_finish_away_name.index(i)]
          finished_all.append(s)

      else:
          finished_all.append("Not applicable")

  finished_all.append("--------")

#   goals = []
  for i in home_players_list:
      if i in home_scorer_name:
          s = ""
          for j in home_scorer_name[i]:
              s += " " + j
          strs = "Scored at " + s
          goals.append(strs)
      else:
          goals.append("Not scored")

  goals.append("---------")
  for i in away_players_list:
      if i in away_scorer_name:
          s = ""
          for j in away_scorer_name[i]:
              s += " " + j
          strs = "Scored at " + s
          goals.append(strs)
      else:
          goals.append("Not scored")
  goals.append("----------")


  for i in home_players_numbers:
    if i in home_fouls_name:
        fouls_player.append(home_fouls_name[i])
    else:
        fouls_player.append("Not applicable")

    if i in home_assists_name:
        assists_player.append(home_assists_name[i])
    else:
        assists_player.append("Not applicable")

    if i in home_offsides_name:
        offside_player.append(home_offsides_name[i])
    else:
        offside_player.append("Not applicable")

    if i in home_int_name:
        inter_player.append(home_int_name[i])
    else:
        inter_player.append("Not applicable")

    if i in home_crs_name:
        crs_player.append(home_crs_name[i])
    else:
        crs_player.append("Not applicable")

    if i in home_shots_name:
        shots_player.append(home_shots_name[i])
    else:
        shots_player.append("Not applicable")

    if i in home_sh_tar_name:
        sh_tar_player.append(home_sh_tar_name[i])
    else:
        sh_tar_player.append("Not applicable")

  fouls_player.append("---------")
  assists_player.append("---------")
  offside_player.append("---------")
  inter_player.append("---------")
  crs_player.append("---------")
  shots_player.append("---------")
  sh_tar_player.append("---------")

  for i in away_players_numbers:
    if i in away_fouls_name:
        fouls_player.append(away_fouls_name[i])
    else:
        fouls_player.append("Not applicable")

    if i in away_assists_name:
        assists_player.append(away_assists_name[i])
    else:
        assists_player.append("Not applicable")

    if i in away_offsides_name:
        offside_player.append(away_offsides_name[i])
    else:
        offside_player.append("Not applicable")

    if i in away_int_name:
        inter_player.append(away_int_name[i])
    else:
        inter_player.append("Not applicable")

    if i in away_crs_name:
        crs_player.append(away_crs_name[i])
    else:
        crs_player.append("Not applicable")

    if i in away_shots_name:
        shots_player.append(away_shots_name[i])
    else:
        shots_player.append("Not applicable")

    if i in away_sh_tar_name:
        sh_tar_player.append(away_sh_tar_name[i])
    else:
        sh_tar_player.append("Not applicable")

  fouls_player.append("---------")
  assists_player.append("---------")
  offside_player.append("---------")
  inter_player.append("---------")
  crs_player.append("---------")
  shots_player.append("---------")
  sh_tar_player.append("---------")


  teams_list.extend(teams)
  team_names.extend([team_home_name, team_away_name])
  locations.extend([location, location])
  team_scores.extend([team_home_score, team_away_score])

  total_attempts.extend([total_attempts_home, total_attempts_away])

  on_targets.extend([on_target_home, on_target_away])

  off_targets.extend([off_target_home, off_target_away])

  blocked.extend([blocked_home, blocked_away])

  woodworks.extend([woodwork_home, woodwork_away])

  corners.extend([corners_home, corners_away])

  offsides.extend([offsides_home, offsides_away])

  possession.extend([possession_home, possession_away])

  passes.extend([passes_home, passes_away])

  passing_accuracy.extend([passing_accuracy_home, passing_accuracy_away])

  passes_completed.extend([passes_completed_home, passes_completed_away])

  distance_covered.extend([distance_covered_home, distance_covered_away])

  balls_recovered.extend([balls_recovered_home, balls_recovered_away])

  tackles.extend([tackles_home, tackles_away])

  clearances.extend([clearances_home, clearances_away])
  blocks.extend([blocks_home, blocks_away])

  yellow_cards.extend([yellow_cards_home, yellow_cards_away])

  red_cards.extend([red_cards_home, red_cards_away])

  fouls_list.extend([fouls_home, fouls_away])
  



#Get the Urls to scrape

We scrape from the Champons League and fbref website

In [0]:
months={'01':'January','02':'February','03':'March','04':'April','05':'May','06':'June','07':'July','08':'August','09':'September','10':'October','11':'November','12':'December'}
def change_date_type(a):
  date=a.split('/')[0]
  date=int(date)
  month=a.split('/')[1]
  month=months[month]
  year=a.split('/')[2]
  string=month+'-'+str(date)+'-'+year
  return string

In [0]:
url='https://fbref.com/en/comps/8/history/Champions-League-Seasons'
page=requests.get(url)
soup = bs.BeautifulSoup(page.content, "html.parser")
start='https://fbref.com'
url_main=[]
text1='-Champions-League-Stats'
for a in soup.find_all('a', href=True):
    if(text1 in a['href']):
      if((start+a['href']) not in url_main):
        url_main.append(start+a['href'])
url_main.pop()

'https://fbref.com/en/comps/8/UEFA-Champions-League-Stats'

In [0]:
match_url=[]
#url='https://fbref.com/en/comps/8/2102/2018-2019-Champions-League-Stats'
for url in url_main:
  page=requests.get(url)
  soup = bs.BeautifulSoup(page.content, "html.parser")
  for a in soup.find_all("div", {"class": "match-detail"}):
    for b in (a.find_all("a",href=True)):
      match_url.append(start+b['href'])
#(match_url)

In [0]:
match1_url=[]
start='https://www.uefa.com/'
find='/uefachampionsleague/match/'
years=['2015','2016','2017','2018','2019']
for year in years:
  url='https://www.uefa.com/uefachampionsleague/history/seasons/'+year+'/matches/'
  page=requests.get(url)
  soup = bs.BeautifulSoup(page.content, "html.parser")
  for a in soup.find_all('a', href=True):
    if (find in a['href']):
      match1_url.append(start+a['href'])
#(match1_url)

In [0]:
urls=[]
for url1 in match1_url:
  page=requests.get(url1)
  soup = bs.BeautifulSoup(page.content, "html.parser")
  a=soup.find("span",{"class":"history-match_score"}).get_text()
  date=change_date_type(a)
  for url in match_url:
    if (date in url):
      a=(url1.split('--')[-1]).lower()
      a=a.rstrip('/')
      c=a.split('-vs-')[0]
      a=a.split('-vs-')[1]
      b=url.lower()
      if ((a in b) or (c in b)):
        #print(url1,url)
        urls.append((url1,url))

In [0]:
from tqdm import tqdm

#Make the Final Dataframes for Player and Teams for years other than 2019

In [0]:
all_players_list = []
teams_list = []
started_all_pd = []
finished_all = []
goals = []
shots_player = []
sh_tar_player = []
assists_player = []
inter_player = []
crs_player = []
fouls_player = []
offside_player = []

team_names = []
locations = []
team_scores = []
total_attempts = []
on_targets = []
off_targets = []
blocked = []
woodworks = []
corners = []
offsides = []
possession = []
passes = []
passing_accuracy = []
passes_completed = []
distance_covered = []
balls_recovered = []
tackles = []
clearances = []
blocks = []
yellow_cards = []
red_cards = []
fouls_list = []

cnt = 1
for match in urls:
    print(cnt)
    print(match[0])
    print(match[1])

    cnt += 1
    get_dataset(match[0],match[1])
    print(len(all_players_list),len(teams_list),len(started_all_pd),len(finished_all),
          len(shots_player),len(sh_tar_player),len(assists_player),len(inter_player),
          len(crs_player), len(fouls_player), len(offside_player))
    
    
  
if True:
  final_dataframe = pd.DataFrame()
  final_dataframe["Players"] = all_players_list
  final_dataframe["Teams"] = teams_list
  final_dataframe["Started_at"] = started_all_pd
  final_dataframe["Finished_at"] = finished_all
  final_dataframe["Goals scored"] = goals
  final_dataframe["Total shots"] = shots_player
  final_dataframe["Shots on Target"] = sh_tar_player
  final_dataframe["Assists"] = assists_player
  final_dataframe["Interceptions"] = inter_player
  final_dataframe["Crosses"] = crs_player
  final_dataframe["Fouls committed"] = fouls_player
  final_dataframe["Offsides"] = offside_player

  print(final_dataframe.head())
    
  teams_df = pd.DataFrame()
  teams_df["Teams"] = team_names
  teams_df["Stadium"] = locations
  teams_df["Goals"] = team_scores
  teams_df["Total Attempts"] = total_attempts
  teams_df["On Target"] = on_targets
  teams_df["Off Target"] = off_targets
  teams_df["Blocked"] = blocked
  teams_df["Woodwork"] = woodworks
  teams_df["Corners"] = corners
  teams_df["Offside"] = offsides
  teams_df["Possession"] = possession
  teams_df["Passes"] = passes
  teams_df["Passing Accuracy"] = passing_accuracy
  teams_df["Passes Completed"] = passes_completed
  teams_df["Distance Covered"] = distance_covered
  teams_df["Balls Recovered"] = balls_recovered
  teams_df["Tackles"] = tackles
  teams_df["Clearances"] = clearances
  teams_df["Blocks"] = blocks
  teams_df["Yellow Cards"] = yellow_cards
  teams_df["Red Cards"] = red_cards
  teams_df["Fouls"] = fouls_list
                        
  print(teams_df.head())


In [0]:
len(crs_player)

In [0]:
final_dataframe.head(40)

In [0]:
final_dataframe.to_csv("players_info.csv",index=False)
teams_df.to_csv("matches_info.csv",index=False)

#Make the dataframes for 2019

In [0]:
matches = [
("https://www.uefa.com/uefachampionsleague/match/2027125--atletico-vs-liverpool/", "https://fbref.com/en/matches/f9a22073/Atletico-Madrid-Liverpool-February-18-2020-Champions-League"),
("https://www.uefa.com/uefachampionsleague/match/2027120--dortmund-vs-paris/","https://fbref.com/en/matches/f778e3b0/Borussia-Dortmund-Paris-Saint-Germain-February-18-2020-Champions-League"),
("https://www.uefa.com/uefachampionsleague/match/2027124--atalanta-vs-valencia/","https://fbref.com/en/matches/85ee0243/Atalanta-Valencia-February-19-2020-Champions-League"),
("https://www.uefa.com/uefachampionsleague/match/2027122--tottenham-vs-leipzig/","https://fbref.com/en/matches/a9417ce9/Tottenham-Hotspur-RB-Leipzig-February-19-2020-Champions-League"),
("https://www.uefa.com/uefachampionsleague/match/2027126--chelsea-vs-bayern/","https://fbref.com/en/matches/cb86e710/Chelsea-Bayern-Munich-February-25-2020-Champions-League"),
("https://www.uefa.com/uefachampionsleague/match/2027123--napoli-vs-barcelona/","https://fbref.com/en/matches/b18968ba/Napoli-Barcelona-February-25-2020-Champions-League"),
("https://www.uefa.com/uefachampionsleague/match/2027127--lyon-vs-juventus/","https://fbref.com/en/matches/e0381a25/Lyon-Juventus-February-26-2020-Champions-League"),
("https://www.uefa.com/uefachampionsleague/match/2027121--real-madrid-vs-man-city/","https://fbref.com/en/matches/de4bd6e8/Real-Madrid-Manchester-City-February-26-2020-Champions-League"),
("https://www.uefa.com/uefachampionsleague/match/2027134--valencia-vs-atalanta/","https://fbref.com/en/matches/43ea56bb/Valencia-Atalanta-March-10-2020-Champions-League"),
("https://www.uefa.com/uefachampionsleague/match/2027128--leipzig-vs-tottenham/","https://fbref.com/en/matches/cff3e534/RB-Leipzig-Tottenham-Hotspur-March-10-2020-Champions-League"),
("https://www.uefa.com/uefachampionsleague/match/2027135--liverpool-vs-atletico/","https://fbref.com/en/matches/d23fe9df/Liverpool-Atletico-Madrid-March-11-2020-Champions-League"),
("https://www.uefa.com/uefachampionsleague/match/2027130--paris-vs-dortmund/","https://fbref.com/en/matches/3ec7f694/Paris-Saint-Germain-Borussia-Dortmund-March-11-2020-Champions-League")
]

In [0]:
all_players_list = []
teams_list = []
started_all_pd = []
finished_all = []
goals = []
shots_player = []
sh_tar_player = []
assists_player = []
inter_player = []
crs_player = []
fouls_player = []
offside_player = []

team_names = []
locations = []
team_scores = []
total_attempts = []
on_targets = []
off_targets = []
blocked = []
woodworks = []
corners = []
offsides = []
possession = []
passes = []
passing_accuracy = []
passes_completed = []
distance_covered = []
balls_recovered = []
tackles = []
clearances = []
blocks = []
yellow_cards = []
red_cards = []
fouls_list = []

cnt = 1

for match in matches:
   print(cnt)
   print(match[0])
   print(match[1])

   cnt += 1
   get_dataset_2019(match[0],match[1])
   print(len(all_players_list),len(teams_list),len(started_all_pd),len(finished_all),
            len(shots_player),len(sh_tar_player),len(assists_player),len(inter_player),
            len(crs_player), len(fouls_player), len(offside_player))

#get_dataset_2019(matches[0][0],matches[0][1])
    
  
if True:
  final_dataframe = pd.DataFrame()
  final_dataframe["Players"] = all_players_list
  final_dataframe["Teams"] = teams_list
  final_dataframe["Started_at"] = started_all_pd
  final_dataframe["Finished_at"] = finished_all
  final_dataframe["Goals scored"] = goals
  final_dataframe["Total shots"] = shots_player
  final_dataframe["Shots on Target"] = sh_tar_player
  final_dataframe["Assists"] = assists_player
  final_dataframe["Interceptions"] = inter_player
  final_dataframe["Crosses"] = crs_player
  final_dataframe["Fouls committed"] = fouls_player
  final_dataframe["Offsides"] = offside_player

  print(final_dataframe.head())
    
  teams_df = pd.DataFrame()
  teams_df["Teams"] = team_names
  teams_df["Stadium"] = locations
  teams_df["Goals"] = team_scores
  teams_df["Total Attempts"] = total_attempts
  teams_df["On Target"] = on_targets
  teams_df["Off Target"] = off_targets
  teams_df["Blocked"] = blocked
  teams_df["Woodwork"] = woodworks
  teams_df["Corners"] = corners
  teams_df["Offside"] = offsides
  teams_df["Possession"] = possession
  teams_df["Passes"] = passes
  teams_df["Passing Accuracy"] = passing_accuracy
  teams_df["Passes Completed"] = passes_completed
  teams_df["Distance Covered"] = distance_covered
  teams_df["Balls Recovered"] = balls_recovered
  teams_df["Tackles"] = tackles
  teams_df["Clearances"] = clearances
  teams_df["Blocks"] = blocks
  teams_df["Yellow Cards"] = yellow_cards
  teams_df["Red Cards"] = red_cards
  teams_df["Fouls"] = fouls_list
                        
  print(teams_df.head())


In [0]:
url = "https://www.uefa.com/uefachampionsleague/fixtures-results/#/rd/2001141/1"
page = requests.get(url)
soup = bs.BeautifulSoup(page.content, "html.parser")